#1. Terceiro projeto do curso de deep learning com pytorch

##2. Importar as bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import torch.nn as nn
import torch
import torch.nn.functional as F
import sklearn
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MaxAbsScaler

###2.1 Instalar skorch 0.8 (está funcionando sem dar erro de dimensionalidade)

In [ ]:
!pip install skorch==0.8.0
from skorch import NeuralNetBinaryClassifier

##3. Deixando o fator randomico de reprodutibilidade padrão em 1

In [ ]:
np.random.seed(1)
torch.manual_seed(1)

##4. Importando o conjunto de dados

In [ ]:
previsores = pd.read_csv('/content/drive/MyDrive/entradas_breast.csv')
predito = pd.read_csv('/content/drive/MyDrive/saidas_breast.csv')
print(predito.shape)
quantidade_unico = np.unique(predito)
print('Classes possíveis: {}'.format(quantidade_unico))

(569, 1)
Classes possíveis: [0 1]


##5. Redefinindo as variaveis para np array

In [ ]:
previsoes = np.array(previsores,dtype='float32')
classes = np.array(predito,dtype='float32').squeeze(1)#tira a dimensão extra do array

##6. Deixando os dados de entrada normalizados

In [ ]:
previsoes_norm = MaxAbsScaler().fit(previsoes)
previsoes_norm = previsoes_norm.transform(previsoes)

##7. Construindo a classe da rede neural

In [ ]:
class classificador(nn.Module):
#Aqui vão ser passado parametros adicionais para a classe
  def __init__(self,activation, neurons, initializer):#inicia o construtor
    super().__init__()#inicializa o construtor da super classe
    
    self.camada0 = nn.Linear(in_features = 30, out_features = neurons)
    initializer(self.camada0.weight)
    self.ativacao0 = activation
    self.dp0 = nn.Dropout(0.2)
    
    self.camada1 = nn.Linear(in_features = 16, out_features = neurons)
    initializer(self.camada1.weight)
    self.ativacao1 = activation
    self.dp1 = nn.Dropout(0.2)
    
    self.camada2 = nn.Linear(in_features=16, out_features=1)
    initializer(self.camada2.weight)
    self.output = nn.Sigmoid()
  
  def forward(self, x):
    x = self.camada0(x)
    x = self.ativacao0(x)
    x = self.dp0(x)
    x = self.camada1(x)
    x = self.ativacao1(x)
    x = self.dp1(x)
    x = self.camada2(x)
    x = self.output(x)
    return x

##8. Definindo o classificador sklearn com skorch

In [ ]:
classificador_sklearn = NeuralNetBinaryClassifier(module=classificador,
                                                  optimizer__weight_decay=0.0001,
                                                  train_split = False
                                                      )

##9. Definindo os parametros do tunning

In [ ]:
parans = {'criterion':[torch.nn.BCELoss,torch.nn.HingeEmbeddingLoss],
          'optimizer':[torch.optim.Adam,torch.optim.SGD],
          'lr':[0.001,0.002,0.003,0.004,0.005],
          'max_epochs':[100,50,20],
          'batch_size':[10,20],
          'module__activation':[torch.relu,torch.sigmoid,torch.tanh],
          'module__neurons':[16,30,40],
          'module__initializer':[torch.nn.init.uniform_,torch.nn.init.zeros_,torch.nn.init.normal_]}

##10. Criar o tunning

In [ ]:
tunning_aval = GridSearchCV(estimator=classificador_sklearn,param_grid=parans,scoring='accuracy',cv=2)
result = tunning_aval.fit(previsoes_norm,classes)

     23       37.1930  0.0548
     24       37.1930  0.0358
     25       37.1930  0.0377
     26       37.1930  0.0452
     27       37.1930  0.0402
     28       37.1930  0.0393
     29       37.1930  0.0341
     30       37.1930  0.0373
     31       37.1930  0.0392
     32       37.1930  0.0381
     33       37.1930  0.0479
     34       37.1930  0.0488
     35       37.1930  0.0452
     36       37.1930  0.0366
     37       37.1930  0.0386
     38       37.1930  0.0373
     39       37.1930  0.0391
     40       37.1930  0.0355
     41       37.1930  0.0348
     42       37.1930  0.0350
     43       37.1930  0.0360
     44       37.1930  0.0408
     45       37.1930  0.0384
     46       37.1930  0.0793
     47       37.1930  0.0430
     48       37.1930  0.0319
     49       37.1930  0.0370
     50       37.1930  0.0439
     51       37.1930  0.0431
     52       37.1930  0.0424
     53       37.1930  0.0441
     54       37.1930  0.0368
     55       37.1930  0.0328
     56   

##11. Mostrar os resultados após o tunning

In [ ]:
print('Melhores Parametros: \n{}\n\nMelhor score: {}'.format(result.best_params_,result.best_score_))